In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from scipy.sparse import csr_matrix, hstack

In [3]:
train = pd.read_table("train.tsv")
test = pd.read_table("test.tsv")

In [4]:
train = train.drop(train[(train.price < 1.0)].index)

In [5]:
NUM_BRANDS = 4000
NUM_CATEGORIES = 1000
NAME_MIN_DF = 10
MAX_FEATURES_ITEM_DESCRIPTION = 50000

In [6]:
def handle_missing_inplace(dataset):
    dataset['category_name'].fillna(value='missing', inplace=True)
    dataset['brand_name'].fillna(value='missing', inplace=True)
    dataset['item_description'].fillna(value='missing', inplace=True)

In [7]:
def cutting(dataset):
    pop_brand = dataset['brand_name'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_BRANDS]
    dataset.loc[~dataset['brand_name'].isin(pop_brand), 'brand_name'] = 'missing'
    pop_category = dataset['category_name'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_CATEGORIES]
    dataset.loc[~dataset['category_name'].isin(pop_category), 'category_name'] = 'missing'

In [8]:
def to_categorical(dataset):
    dataset['category_name'] = dataset['category_name'].astype('category')
    dataset['brand_name'] = dataset['brand_name'].astype('category')
    dataset['item_condition_id'] = dataset['item_condition_id'].astype('category')

In [9]:
nrow_train = train.shape[0]
y = np.log1p(train["price"])
merge: pd.DataFrame = pd.concat([train, test])
submission: pd.DataFrame = test[['test_id']]

In [10]:
handle_missing_inplace(merge)

In [11]:
cutting(merge)

In [12]:
to_categorical(merge)

In [16]:
print train["item_description"][2]

'Adorable top with a hint of lace and a key hole in the back! The pale pink is a 1X, and I also have a 3X available in white!'

In [13]:
merge.head(25)

,brand_name,category_name,item_condition_id,item_description,name,price,shipping,test_id,train_id
0,missing,Men/Tops/T-shirts,3,No description yet,MLB Cincinnati Reds T Shirt Size XL,10.0,1,NaN,0.0
1,Razer,Electronics/Computers & Tablets/Components & P...,3,This keyboard is in great condition and works ...,Razer BlackWidow Chroma Keyboard,52.0,0,NaN,1.0
2,Target,Women/Tops & Blouses/Blouse,1,Adorable top with a hint of lace and a key hol...,AVA-VIV Blouse,10.0,1,NaN,2.0
3,missing,Home/Home Décor/Home Décor Accents,1,New with tags. Leather horses. Retail for [rm]...,Leather Horse Statues,35.0,1,NaN,3.0
4,missing,Women/Jewelry/Necklaces,1,Complete with certificate of authenticity,24K GOLD plated rose,44.0,0,NaN,4.0
5,missing,Women/Other/Other,3,"Banana republic bottoms, Candies skirt with ma...",Bundled items requested for Ruie,59.0,0,NaN,5.0
6,Acacia Swimwear,Women/Swimwear/Two-Piece,3,Size small but straps slightly shortened to fi...,Acacia pacific tides santorini top,64.0,0,NaN,6.0
7,Soffe,Sports & Outdoors/Apparel/Girls,3,You get three pairs of Sophie cheer shorts siz...,Girls cheer and tumbling bundle of 7,6.0,1,NaN,7.0
8,Nike,Sports & Outdoors/Apparel/Girls,3,Girls Size small Plus green. Three shorts total.,Girls Nike Pro shorts,19.0,0,NaN,8.0
9,missing,Vintage & Collectibles/Collectibles/Doll,3,I realized his pants are on backwards after th...,Porcelain clown doll checker pants VTG,8.0,0,NaN,9.0


In [13]:
cv = CountVectorizer(min_df=NAME_MIN_DF)
X_name = cv.fit_transform(merge['name'])

In [14]:
cv = CountVectorizer()
X_category = cv.fit_transform(merge['category_name'])

In [15]:
merge.head()

,brand_name,category_name,item_condition_id,item_description,name,price,shipping,test_id,train_id
0,missing,Men/Tops/T-shirts,3,No description yet,MLB Cincinnati Reds T Shirt Size XL,10.0,1,NaN,0.0
1,Razer,Electronics/Computers & Tablets/Components & P...,3,This keyboard is in great condition and works ...,Razer BlackWidow Chroma Keyboard,52.0,0,NaN,1.0
2,Target,Women/Tops & Blouses/Blouse,1,Adorable top with a hint of lace and a key hol...,AVA-VIV Blouse,10.0,1,NaN,2.0
3,missing,Home/Home Décor/Home Décor Accents,1,New with tags. Leather horses. Retail for [rm]...,Leather Horse Statues,35.0,1,NaN,3.0
4,missing,Women/Jewelry/Necklaces,1,Complete with certificate of authenticity,24K GOLD plated rose,44.0,0,NaN,4.0


In [16]:
tv = TfidfVectorizer(max_features=MAX_FEATURES_ITEM_DESCRIPTION, ngram_range=(1, 3), stop_words='english')
X_description = tv.fit_transform(merge['item_description'])

In [17]:
lb = LabelBinarizer(sparse_output=True)
X_brand = lb.fit_transform(merge['brand_name'])

In [18]:
X_dummies = csr_matrix(pd.get_dummies(merge[['item_condition_id', 'shipping']], sparse=True).values)

In [19]:
sparse_merge = hstack((X_dummies, X_description, X_brand, X_name)).tocsr()

In [20]:
X = sparse_merge[:nrow_train]
X_test = sparse_merge[nrow_train:]

In [39]:
model = Ridge(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=100,
      normalize=False, random_state=80, solver='auto', tol=0.01)

In [40]:
model.fit(X, y)

Ridge(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=100,
   normalize=False, random_state=80, solver='auto', tol=0.01)

In [41]:
model.score(X, y)

0.63213261886333594

In [42]:
predsR = model.predict(X=X_test)

In [ ]:
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size = 0.15, random_state = 80) 
d_train = lgb.Dataset(train_X, label=train_y, max_bin=8192)
d_valid = lgb.Dataset(valid_X, label=valid_y, max_bin=8192)
watchlist = [d_train, d_valid]
params = {
        'learning_rate': 0.7,
        'application': 'regression',
        'max_depth': 3,
        'num_leaves': 60,
        'verbosity': -1,
        'metric': 'RMSE',
        'data_random_seed': 1,
        'bagging_fraction': 1,
        'nthread': 4
}
model = lgb.train(params, train_set=d_train, num_boost_round=7500, valid_sets=watchlist, \
    early_stopping_rounds=500, verbose_eval=500) 
predsL = model.predict(X_test)

C:\Users\diaschen\AppData\Local\Continuum\anaconda\lib\site-packages\lightgbm\basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)
C:\Users\diaschen\AppData\Local\Continuum\anaconda\lib\site-packages\lightgbm\basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))


Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 0.499869	valid_1's rmse: 0.504351
[1000]	training's rmse: 0.47785	valid_1's rmse: 0.486884
[1500]	training's rmse: 0.466246	valid_1's rmse: 0.479368


In [ ]:
print('finish')

In [ ]:
train_X2, valid_X2, train_y2, valid_y2 = train_test_split(X, y, test_size = 0.1, random_state = 101) 
d_train2 = lgb.Dataset(train_X2, label=train_y2, max_bin=8192)
d_valid2 = lgb.Dataset(valid_X2, label=valid_y2, max_bin=8192)
watchlist2 = [d_train2, d_valid2]
params2 = {
        'learning_rate': 0.95,
        'application': 'regression',
        'max_depth': 3,
        'num_leaves': 140,
        'verbosity': -1,
        'metric': 'RMSE',
        'data_random_seed': 2,
        'bagging_fraction': 0.5,
        'nthread': 4
}

model = lgb.train(params2, train_set=d_train2, num_boost_round=5000, valid_sets=watchlist2, \
early_stopping_rounds=50, verbose_eval=500) 
predsL2 = model.predict(X_test)

In [ ]:
print('finish')

In [ ]:
preds = predsR*0.3 + predsL*0.35 + predsL2*0.35

In [68]:
test["price"] = np.expm1(preds)
test[["test_id", "price"]].to_csv("submi.csv", index = False)